In [1]:
import pandas as pd
import numpy as np
import gc
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
#from minepy import MINE
from mlxtend.regressor import StackingRegressor
from collections import defaultdict

E:\python-ana\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_set = pd.read_excel('D://训练_20180117.xlsx')
test_set = pd.read_excel('D://测试A_20180117.xlsx')
submit = pd.read_csv('D://answer_sample_a_20180117.csv', names=['id', 'Y'])

y = data_set[['Value']]

data_set = data_set.drop(['Value'], axis=1)
Le = LabelEncoder()

In [3]:
input_data = pd.concat([data_set, test_set], ignore_index=True)
input_data = input_data.drop(['ID'], axis=1)

In [4]:
col = pd.DataFrame(input_data.ix[:, 2:].columns, columns=['col'])
TOOL_ID_col_dict = defaultdict(lambda : 0)
for index,row in col.iterrows():
    info=row.col.split('X')[0]
    TOOL_ID_col_dict[info] += 1

TOOL_ID_col_dict

E:\python-ana\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


defaultdict(<function __main__.<lambda>>,
            {'210': 228,
             '220': 179,
             '300': 21,
             '310': 170,
             '311': 184,
             '312': 679,
             '330': 889,
             '340': 139,
             '344': 398,
             '360': 1030,
             '400': 183,
             '420': 173,
             '440A': 213,
             '520': 382,
             '750': 1030,
             'Chamber': 1,
             'Chamber ID': 1,
             'ERROR:#N/A': 1,
             'ERROR:#N/A (#1)': 1,
             'ERROR:#N/A (#2)': 1,
             'ERROR:#N/A (#3)': 1,
             'ERROR:#N/A_1': 1,
             'ERROR:#N/A_1 (#1)': 1,
             'ERROR:#N/A_1 (#2)': 1,
             'ERROR:#N/A_10': 1,
             'ERROR:#N/A_11': 1,
             'ERROR:#N/A_12': 1,
             'ERROR:#N/A_13': 1,
             'ERROR:#N/A_14': 1,
             'ERROR:#N/A_15': 1,
             'ERROR:#N/A_16': 1,
             'ERROR:#N/A_17': 1,
             'ERROR

In [5]:
#删除全空字段
null_col = []
for col in data_set.columns:
    if data_set[col].isnull().all() == True:
        null_col.append(col)
        
input_data.drop(null_col, axis=1, inplace=True)    
input_data.drop(['220X150', '220X151'], axis=1, inplace=True)    

In [6]:
#转换obj字段
obj_col = []
for col in input_data.columns:
    if input_data[col].dtypes == object:
        obj_col.append(col)
        
for i in range(len(obj_col)):
    Le.fit(input_data[obj_col[i]].unique())
    input_data[obj_col[i]] = Le.transform(input_data[obj_col[i]])

In [7]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.interpolate())      
    for line in data.columns:
        if len(data[line].unique()) == 1:
            data = data.drop([line], axis=1)    
    return data

input_data = drop_col(input_data)

In [8]:
#剔除较大值
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

input_data = date_cols(input_data)

In [9]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 3 * (Q3 - Q1)
        max_v = Q3 + 3 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].mean()
           
    return data

input_data = quantile_dropout(input_data)
input_data.fillna(input_data.mean(), inplace=True)
input_data = drop_col(input_data)

E:\python-ana\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
#剔除小方差字段
VT=VarianceThreshold(threshold=0.5)
input_data = pd.DataFrame(VT.fit_transform(input_data))

In [11]:
train = input_data.ix[:799, :]
test = input_data.ix[800:, :]
clf_gt = ensemble.GradientBoostingRegressor(learning_rate=0.01, n_estimators=160, random_state=0)
clf_gt.fit(train, y)

model = SelectFromModel(clf_gt, prefit=True) 
train = pd.DataFrame(model.transform(train))
test = pd.DataFrame(model.transform(test))

E:\python-ana\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
E:\python-ana\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
train.to_csv('D://up_to_Pai_train.csv', index=False)
test.to_csv('D://up_to_Pai_test.csv', index=False)

In [13]:
def norm(data_X):
    for j in [col for col in data_X]:
        t_max=data_X[j].max()
        t_min=data_X[j].min()
        down = 1 if (t_max-t_min == 0) else (t_max-t_min)
        data_X[j]=(data_X[j]-t_min)/down

In [14]:
norm(train)
norm(test)

In [15]:
f_num=205
REGULARIZATION_RATE = 0.0001
LEARNING_RATE_BASE = 0.005
LEARNING_RATE_DECAY = 0.99
training_steps=700
HIDDEN_SIZE=40
HIDDEN_1_SIZE=10
OUTPUT_SIZE=1

In [17]:
with tf.variable_scope("dnn",reuse=tf.AUTO_REUSE):
    X = tf.placeholder("float",[None,f_num,])
    Y = tf.placeholder("float", [None,OUTPUT_SIZE])
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    global_step = tf.Variable(0, trainable=False)
    
    lay_1 = tf.Variable(tf.random_normal([f_num, HIDDEN_SIZE], mean=0.0, stddev=1.0))
    lay_2 = tf.Variable(tf.random_normal([HIDDEN_SIZE,HIDDEN_1_SIZE],mean=0.0,stddev=1.0))
    lay_3 = tf.Variable(tf.random_normal([HIDDEN_1_SIZE,OUTPUT_SIZE],mean=0.0,stddev=1.0))
    #lay_1 = tf.Variable(tf.ones([f_num, HIDDEN_SIZE]))
    #lay_2 = tf.Variable(tf.ones([HIDDEN_SIZE,HIDDEN_1_SIZE]))
    #lay_3 = tf.Variable(tf.ones([HIDDEN_1_SIZE,OUTPUT_SIZE]))
    b_1 = tf.Variable(tf.zeros([HIDDEN_SIZE]),dtype="float32",name="bias_1")
    b_2 = tf.Variable(tf.zeros([HIDDEN_1_SIZE]),dtype="float32",name="bias_2")
    b_3 = tf.Variable(tf.zeros([OUTPUT_SIZE]),dtype="float32",name="bias_3")
    
    lay_out1 = tf.nn.sigmoid(tf.matmul(tf.reshape(X,[-1,f_num]), lay_1) + b_1)
    lay_out2 = tf.nn.sigmoid(tf.matmul(tf.reshape(lay_out1,[-1,HIDDEN_SIZE]), lay_2) + b_2)
    lay_out3 = tf.nn.sigmoid(tf.matmul(tf.reshape(lay_out2,[-1,HIDDEN_1_SIZE]), lay_3) + b_3)
    
    y_=tf.add(tf.matmul(tf.reshape(lay_out2,[-1,HIDDEN_1_SIZE]),lay_3),b_3)
    tf.add_to_collection('losses', regularizer(lay_1))
    tf.add_to_collection('losses', regularizer(lay_2))
    tf.add_to_collection('losses', regularizer(lay_3))

    di=y_-Y
    loss_op = tf.reduce_mean(tf.square(y_ - Y))
    loss=tf.sqrt(loss_op)
    learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            200, LEARNING_RATE_DECAY,
            staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss_op,global_step=global_step)
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

In [25]:
def train_(train_data,train_label,test_data,test_label):
    with tf.Session() as sess:
        # Run the initializer
        sess.run(init)
        #test_X=[]
        #test_y=[]
        #for i in range(len(date_list)):
        #test_X=(np.array(date_list[0],dtype=np.float32)[0:5])
        #test_y=(np.array([ [sale_amt_list[0].iloc[i]] for i in range(5)]))
        for step in range(len(train_data)-5):
            train_X=np.array(train_data.ix[step:step+5],dtype=np.float32)
            train_y=np.array(train_label.ix[step:step+5],dtype=np.float32)
            step=step+5
            #test_X=np.array(data_X_test,dtype=np.float32)
            sess.run(train_op, feed_dict={X: train_X, Y: train_y})
                
            #if step % 10 == 0:
            #    pass
                    #print(sess.run(loss,feed_dict={X:np.array(date_list[0].iloc[5],dtype=np.float32), Y: np.array(sale_list[0].iloc[-1],dtype=np.float32)}))
        print(sess.run([loss],feed_dict={X:test_data, Y: test_label}))
        pre=sess.run(y_,feed_dict={X:train})
        result=sess.run(y_,feed_dict={X:test})

In [26]:
for i in range(10):
    step = 80
    index = i * 80
    test_data_tmp = train.ix[ index:index+step ]
    test_y_tmp = y.ix[ index:index+step ]
    train_data_tmp = pd.concat([train.ix[0:index],train.ix[index+step:]])
    train_y_tmp = pd.concat([y.ix[0:index],y.ix[index+step:]])
    train_(train_data_tmp,train_y_tmp,test_data_tmp,test_y_tmp)

E:\python-ana\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
E:\python-ana\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


[0.29034424]
[0.25600332]
[0.36373174]
[0.2383952]
[0.21183531]
[0.29053044]
[0.32217667]
[0.28598034]
[0.2880584]
[0.36296573]


In [19]:
pre

array([[ 2.86559224],
       [ 3.17306352],
       [ 3.06832242],
       [ 3.13019586],
       [ 3.19660258],
       [ 2.78832078],
       [ 3.10694599],
       [ 2.95389438],
       [ 3.08092046],
       [ 3.04077458],
       [ 2.99637651],
       [ 2.83566475],
       [ 2.95633984],
       [ 2.85910296],
       [ 2.94745588],
       [ 3.0601244 ],
       [ 2.88289499],
       [ 3.26671553],
       [ 2.90545535],
       [ 3.15085483],
       [ 2.90768743],
       [ 3.07473087],
       [ 3.47133946],
       [ 2.83167887],
       [ 2.8089447 ],
       [ 2.86200595],
       [ 2.80216241],
       [ 2.84136009],
       [ 2.84062576],
       [ 2.84801316],
       [ 2.91559839],
       [ 2.85125136],
       [ 2.85608888],
       [ 2.85812473],
       [ 2.80125213],
       [ 2.94756246],
       [ 2.83108664],
       [ 2.92000747],
       [ 2.92216039],
       [ 3.10941005],
       [ 3.01328969],
       [ 2.9861927 ],
       [ 2.8898859 ],
       [ 2.82857633],
       [ 2.78666353],
       [ 3

In [20]:
y

,Value
0,2.814025
1,2.782955
2,2.725973
3,2.794768
4,2.787539
5,2.569175
6,2.965824
7,2.965824
8,2.773921
9,2.773921


In [ ]:
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.5
param['seed'] = 0
xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_boost_round=4200, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0]) 

In [ ]:
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.5
param['seed'] = 0
xgbTrain = xgb.DMatrix(train, label=y)
xgbTest = xgb.DMatrix(test)
modle = xgb.train(param, xgbTrain, num_boost_round=4200)
submit['Y'] = modle.predict(xgbTest)

In [ ]:
submit.to_csv('复赛-submission{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)